# 1样本与数据说明
## 1.1样本
    此次是参照宫晓莉（2020）文献中样本的选择，对中国在2007年之前上市的所有银行进行研究，主要是为了包括08年次贷危机这个时间段进行研究，14家银行分别为，平安银行(000001.SZ), 浦发银行(600000.SH), 华夏银行(600015.SH), 民生银行(600016.SH), 招商银行(600036.SH), 兴业银行(601166.SH), 交通银行(601328.SH), 工商银行(601398.SH), 建设银行(601939.SH), 中国银行(601988.SH), 中信银行(601998.SH), 宁波银行(002142.SZ), 南京银行(601009.SH), 北京银行(601169.SH)。

    此外关于金融system的定义为所有上市的银行，即会动态包括更新上市的新银行，这样会可以使得对金融system的定义没有偏误，比如农业银行（2010年上市），苏州银行（2019年上市）等其他银行，加上原来研究的14家银行，在最近的金融系统中一共有40家上市银行（即截止2021年8月6号，中国一共有40家上市的银行，这些银行构成了新的金融系统）
    
    样本的时间跨度为2007年至今


## 1.2数据说明

    数据的频率单位为周，即t-1期和t期的时间间隔为一周
    以下所有数据都来源于Wind
    
    银行层面的数据
    1.银行股价的周度涨幅（即每周的收益率）
    2.金融system的周度市值涨跌幅，即所有上市银行的市值总和的变动情况，即为金融系统的收益率
    
    宏观变量的数据（参考欧阳资生，2020，中国管理科学）
    1.股票市场收益率，选择上证综指每周的收益率数据，源于Wind的指数部分的指数行情序列-上证综合指数
    2.股票市场的波动率，选取上证综指周波动率表示，源于Wind的指数部分的制指数行情序列-上证综合指数-周行情-周振幅
    3.短期流动性利差，用6个月期shibor（上海银行间同业拆放利率）和6个月国债利率之差表示，shibor可以从Wind的债券-多维数据-利率走势分析-货币市场-拆借利率中获得，国债的数据也可以从债券-多维数据-利率走势分析-债券市场-到期收益率（中证）-国债收益率曲线中获得，下面的其他债券收益率都可以从债券市场-到期收益率（中证）中获得
    4.利率风险，选取6个月国债利率，用其前后期到期收益率的差表示，即t期和t-1期的差
    5.利率期限结构，用t时刻6个月国债利率与10年国债利率之差，减去t-1时期的6个月的国债利率与10年期国债利率之差表示
    6.违约风险，用t时期10年期国债利率与10年期AA+级信用登记企业债之差，减去t-1时期10年期国债利率与10年期AA+级信用登记企业债之差表示
    7.房地产收益率，选取申万房地产指数每周收盘价表示，数据来源于Wind数据库
    
    CoVaR原文宏观变量的选择（这是对应美国市场的宏观变量）
    1.The weekly equity market return from CRSP.
    2.VIX, which captures the implied volatility in the stock market reported by the Chicago Board Options Exchange
    3.A short term “liquidity spread,” defined as the difference between the three-month repo rate and the three-month bill rate. This liquidity spread measures short-term liquidity risk.
    4.The change in the three-month Treasury bill rate from the Federal Reserve Board’s H.15. 
    5.The change in the slope of the yield curve, measured by the yield spread between the ten-year Treasury rate and the three-month bill rate obtained from the Federal Reserve Board’s H.15 release.
    6.The change in the credit spread between BAA-rated bonds and the Treasury rate (with the same maturity of ten years) from the Federal Reserve Board’s H.15 release.
    7.The weekly real estate sector return in excess of the market return
    
    宏观变量的选取主要是为了求Time-varying的CoVaR，另外CoVaR原文的宏观变量选择主要是美国市场的宏观变量选取，为了转换到中国银行宏观变量的选取，这里我们参考了嵌入网络舆情指数的中国金融机构系统性风险传染效应研究（欧阳资生，2020，中国管理科学），因为目前中文文献有标明宏观变量的选择的暂时只有这一篇。

# 代码编写

导入一些必要的包

In [250]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import gmean
import math
from pandas import Series, DataFrame
import time
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from pandas.core.frame import DataFrame

## 2.1读取文件数据

读取14家上市银行的数据

In [141]:
#将14家银行的excel名称列成一个列表
bank14=['000001.SZ.xlsx','600000.SH.xlsx','600015.SH.xlsx','600016.SH.xlsx','600036.SH.xlsx','601166.SH.xlsx',\
        '601328.SH.xlsx','601398.SH.xlsx','601939.SH.xlsx','601988.SH.xlsx','601998.SH.xlsx','002142.SZ.xlsx',\
       '601009.SH.xlsx','601169.SH.xlsx']
#记录想要研究的银行的数目
bank_research_len=len(bank14)

bank14_name=[] #用来记录14家银行的名称
bank14_df=[] #用来记录14家银行的df数据
bank14_ret_df=[]
#开始读取14家银行的数据
for i in range(bank_research_len):
    path_name='bank_data/'+bank14[i] #这是读取文件的路径 如果报错的话 查看一下路径对不对
    excel_read=pd.read_excel(path_name,encoding='UTF-8')
    #因为Wind导出的数据后面几列有数据来源：Wind的这几行，会产生NaN值，所以删掉
    excel_read=excel_read.dropna(axis=0,how='any') 
    #记录导入的14家银行的名称
    bank14_name.append(excel_read['简称'].iloc[0])
    bank14_df.append(excel_read) #将该银行数据加入df列表

In [140]:
print(list(bank14_df[0]))

['代码', '简称', '日期', '涨跌(元)', '涨跌幅(%)', '总市值(元)']


继续读取其余在07年以后上市银行的数据，比如农业银行（2010年上市），苏州银行（2019年上市）等，这里面一共有26家银行，加上之前的14家，一共40家银行

In [30]:
#将这些银行的excel名称也列成一个列表
bank_rest=['601288.SH.xlsx','002807.SZ.xlsx','002839.SZ.xlsx','002936.SZ.xlsx','002948.SZ.xlsx','002958.SZ.xlsx',\
          '002966.SZ.xlsx','600908.SH.xlsx','600919.SH.xlsx','600926.SH.xlsx','600928.SH.xlsx','601658.SH.xlsx',\
          '601077.SH.xlsx','601128.SH.xlsx','601818.SH.xlsx','601187.SH.xlsx','601229.SH.xlsx','601528.SH.xlsx',\
          '601577.SH.xlsx','601665.SH.xlsx','601838.SH.xlsx','601860.SH.xlsx','601963.SH.xlsx','601997.SH.xlsx',\
          '603323.SH.xlsx','601916.SH.xlsx']
bank_rest_len=len(bank_rest) #计算这些银行的数目
bank_rest_name=[]
bank_rest_df=[]
#开始读取这些银行的数据
for i in range(bank_rest_len):
    path_name='bank_data/'+bank_rest[i] #这是读取文件的路径 如果报错的话 查看一下路径对不对
    excel_read=pd.read_excel(path_name,encoding='UTF-8')
    excel_read=excel_read.dropna(axis=0,how='any')
    bank_rest_name.append(excel_read['简称'].iloc[0])
    bank_rest_df.append(excel_read) #将该银行数据加入df列表


计算金融system的总市值，即所有上市银行的总市值

In [50]:
bank_system_df=bank14_df+bank_rest_df
bank_system_name=bank14_name+bank_rest_name
bank_system_len=len(bank_system_df)
#计算system的总市值，即所有银行总市值
bank_sys_merge=bank_system_df[0][['日期','总市值(元)']] #这里列是总市值其实指的是excel里面单个银行的总市值
bank_sys_merge.columns=['日期',bank_system_name[0]] #把列名改一下，这样求系统总市值的时候容易区分
for i in range(1,bank_system_len):
    bank_temp=bank_system_df[i][['日期','总市值(元)']]
    bank_temp.columns=['日期',bank_system_name[i]]
    bank_sys_merge=pd.merge(bank_sys_merge,bank_temp,how='outer',on='日期')

#bank_sys_merge已经包括了所有银行的市值了，但是由于是outer，里面还有NaN值，需要变成0
#比如有些银行在2019年才上市，所以前面的值是缺失的
bank_sys_merge.fillna(0, inplace=True)
#然后对每个银行的市值求和，得到system的总市值
bank_sys_merge['sum']=bank_sys_merge.iloc[:,1:bank_system_len+1].sum(axis=1) #即除掉日期以外的列求和


读取宏观变量的数据，后续要改程序的话，由于不同宏观变量的导出方式不同，可能要注意调整一下，而且有时不同的Wind导出来的格式可能也会存在差异，比如有些excel出来的前面2年都是变量名字（列名），有些是只有前面1行是列名

In [97]:
#宏观变量的文件名
marco_excel_name=['国债与企业债利率-20210806.xlsx','上证收益率和波动率.xlsx','SHIBOR0806.xlsx','房地产行情数据.xlsx']

marco_excel_len=len(marco_excel_name)

marco_df=[] #用来记录14家银行的df数据
#开始读取14家银行的数据
for i in range(marco_excel_len):
    path_name='marco_data/'+marco_excel_name[i] #这是读取文件的路径 如果报错的话 查看一下路径对不对
    excel_read=pd.read_excel(path_name,encoding='UTF-8')
    excel_read=excel_read.drop(axis=0, index=0, columns=None, inplace=False)
    #因为Wind导出的数据后面几列有数据来源：Wind的这几行，会产生NaN值，所以删掉
    excel_read=excel_read.dropna(axis=0,how='any') 
    marco_df.append(excel_read) #将该银行数据加入df列表

#因为宏观变量的数据不像银行数据批量导出的那么整齐，不同excel中变量数目也不同，所以这几个宏观变量要单独处理，没有for循环那么方便了
#后续要改程序的话，由于不同宏观变量的导出方式不同，可能要注意调整一下

#这里需要调整一下宏观变量的列名，不然到时不好按日期把dataframe进行合并
#调整一下国债与企业债数据的列名
#下面分别表示日期，AA企业债10年利率，AA+企业债10年利率，国债10年利率，国债6个月利率
marco_df[0].columns=['日期','AA_10y','AA+_10y','trea_10y','trea_6m']
#调整上证收益率和波动率的列名
marco_df[1].columns=['日期','SH_retu','SH_vol']
#调整上海银行同业拆借利率的列名
marco_df[2].columns=['日期','shibor']
#调整房地产数据的列名
marco_df[3].columns=['日期','house']


这里定义一个函数，用来处理日期的，Wind导出的日期主要有两种格式，一种是2009-08-03格式，另外一种是20090803格式，我们需要把格式给统一起来。另外由于我们计算的是周度单位，所以方便起见，我们统一成的格式为200835，意思为2008年的第35周

In [186]:
def getweek(data):
    year_week=[]#,b='-';
    for i in data:
        if type(i) is float:
            i=int(i)
        i=str(i)
        #print(i[4])
        if i[4] is '-': #这个是用来处理2009-08-03类的
            year_record=i[0:4];mon_record=i[5:7];day_record=i[8:10];
            date_temp=year_record+mon_record+day_record;
            #print(date_temp)
            week_record=datetime.strptime(date_temp,"%Y%m%d").strftime("%W")
            year_week_temp=year_record+week_record
            year_week_temp=int(year_week_temp)
            year_week.append(year_week_temp)
        else:
            year_record=i[0:4]; #这个是用来处理20090803类的
            date_temp=i[0:8]
            week_record=datetime.strptime(date_temp,"%Y%m%d").strftime("%W")
            #week_record=datetime.strptime(date_temp,"%Y%m%d").strftime("%W")
            year_week_temp=year_record+week_record
            year_week_temp=int(year_week_temp)
            year_week.append(year_week_temp)
    return year_week




In [99]:
#对宏观变量的日期进行处理
for i in marco_df:
    year_week=getweek(i['日期'])
    i.insert(0,'year_week',year_week) #插入200835样式
    del i['日期'] #去掉原本日期
  

In [118]:
#把所有宏观变量合在一张表上
marco_merge=marco_df[0]
for i in range(1,marco_excel_len):
    marco_merge=pd.merge(marco_merge,marco_df[i],how='inner',on='year_week') #这里宏观变量序列取交集（有数据的部分）

In [119]:
#定义一个差分的函数，即t期减去t-1期，因为后面经常要对时间差分
def diff1(data):
    data_diff=data-data.shift(1)
    return data_diff

In [133]:
#对宏观变量按日期升序进行重新排列
marco_merge_ascend=marco_merge.sort_index(axis=0,by='year_week',ascending=True)

#对宏观变量进行处理，得到我们需要的变量
#计算流动性利差，用6个月期shibor（上海银行间同业拆放利率）和6个月国债利率之差表示
liquid_spread=marco_merge_ascend['shibor']-marco_merge_ascend['trea_6m']
marco_merge_ascend.insert(2,'liquid_spread',liquid_spread)

#利率风险，选取6个月国债利率，用其前后期到期收益率的差表示，即t期和t-1期的差
int_risk=diff1(marco_merge_ascend['trea_6m'])
marco_merge_ascend.insert(2,'int_risk',int_risk)

#利率期限结构，用t时刻6个月国债利率与10年国债利率之差，减去t-1时期的6个月的国债利率与10年期国债利率之差表示
term_structure=diff1(marco_merge_ascend['trea_6m']-marco_merge_ascend['trea_10y'])
marco_merge_ascend.insert(2,'term_structure',term_structure)

#违约风险，用t时期10年期国债利率与10年期AA+级信用登记企业债之差，减去t-1时期10年期国债利率与10年期AA+级信用登记企业债之差表示
default_risk=diff1(marco_merge_ascend['AA_10y']-marco_merge_ascend['trea_10y'])
marco_merge_ascend.insert(2,'default_risk',default_risk)

#提取出后续需要用到的宏观变量
marco_var=marco_merge_ascend[['year_week','SH_retu','SH_vol','liquid_spread','int_risk','term_structure'\
                             ,'default_risk','house']]

计算金融system的收益率

In [136]:
#金融系统的收益率
bank_sys_ret=diff1(bank_sys_merge['sum'])/bank_sys_merge['sum']
bank_sys_merge.insert(2,'bank_sys_ret',bank_sys_ret)
bank_sys_merge=bank_sys_merge.dropna(axis=0,how='any')

#先把日期也处理掉，换成年+周的格式
year_week=getweek(bank_sys_merge['日期'])
bank_sys_merge.insert(0,'year_week',year_week) #插入200835样式
del bank_sys_merge['日期'] #去掉原本日期

#把金融系统的收益率添加到宏观变量里面去
bank_sys_merge_select=bank_sys_merge[['year_week','bank_sys_ret']]
marco_var=pd.merge(marco_var,bank_sys_merge_select,how='inner',on='year_week')


把14家研究的银行的收益率放在一表中

In [210]:
#每个银行的数据加上宏观变量
bank14_add_marco=[]
for i in range(bank_research_len):
    bank_single=bank14_df[i][['日期','涨跌幅(%)']]
    year_week_2=getweek(bank_single['日期'])
    bank_single.insert(0,'year_week',year_week_2)
    del bank_single['日期']
    bank_single2=bank_single.rename(columns={'涨跌幅(%)':'retu'})
    bank14_add_marco.append(pd.merge(bank_single2,marco_var,how='inner',on='year_week'))

#因此bank14_add_marco这个列表记录后面每个机构拿去做分位数回归的数据

In [213]:
print(bank14_add_marco[1].shape[0])

701



## 2.2计算$\Delta CoVaR$
计算$\Delta$CoVaR的公式为
$$
X_{t}^{i}=\alpha^{i}_{q}+\gamma^{i}_{q} M_{t-1}+\varepsilon_{t}^{i}
$$
$$
X_{t}^{\text {system }}=\alpha_{q}^{\text {system } \mid i}+\beta_{q}^{\text {system } \mid i} X_{t}^{i}+\gamma_{q}^{\text {system } \mid i} M_{t-1}+\varepsilon_{t}^{\text {system } \mid i}
$$
    可以通过上式的回归拟合得到，分位数回归的系数，进而进行预测
    把$X_{t}^{i}$替换成$V a R_{t}^{i}(q)$进行预测对应的分位数
    先计算出$V a R_{t}^{i}(q)$的值
$$
V a R_{t}^{i}(q)=\alpha_{q}^{i}+\gamma_{q}^{i} M_{t-1}
$$

这样就可以计算出$\operatorname{CoVaR}_{t}^{i}(q)$
再用中等情况和异常情况做差
$$
\Delta C o V a R_{q}^{s y s t e m \mid i}=\hat{\beta}_{q}^{i}\left(V a R_{q}^{i}-V a R_{50 \%}^{i}\right)
$$
这样就可以得到每家银行的$\Delta CoVaR$了

注意到宏观变量和X的时间分别为t-1和t期，是隔一期的，所以要调整宏观变量数据移动一期



#### 下面进行程序的编写

data_address函数主要是将宏观变量变成t-1期，对齐retu和bank_sys_ret的t期

In [227]:
def data_address(data1):
    data1[['SH_retu','SH_vol','liquid_spread','int_risk','term_structure','default_risk','house']]\
    =data1[['SH_retu','SH_vol','liquid_spread','int_risk','term_structure','default_risk','house']].shift(1)
    new_data=data1.dropna(axis=0,how='any') #删掉因为shift出现的NaN的行
    new_data=pd.DataFrame(new_data,dtype=np.float)
    return new_data


计算$\Delta CoVaR$的函数

In [279]:
def covar(data):
    data=data_address(data) #data_address主要是将宏观变量t-1期一位
    #print(data)
    #单个机构收益率与宏观宏观变量做回归，对应公式第一个式子，主要为了求得分位数回归系数
    model_1=smf.quantreg('retu ~ SH_retu + SH_vol + liquid_spread + int_risk + term_structure + default_risk + house',data)
    #计算50%的分位数回归系数
    res1=model_1.fit(q=0.5)
    #计算5%的分位数回归系数
    res2=model_1.fit(q=0.05)
    #计算出机构i的50分位数收益率
    Var_50=res1.params['SH_retu']*data.iloc[-1,2]+res1.params['SH_vol']*data.iloc[-1,3]+res1.params['liquid_spread']*data.iloc[-1,4]\
    +res1.params['int_risk']*data.iloc[-1,5]+res1.params['term_structure']*data.iloc[-1,6]+\
    res1.params['default_risk']*data.iloc[-1,7]+res1.params['house']*data.iloc[-1,8]+res1.params['Intercept']
    #print(Var_50)
    #计算出机构i的5分位数收益率
    Var_5=res2.params['SH_retu']*data.iloc[-1,2]+res2.params['SH_vol']*data.iloc[-1,3]+res2.params['liquid_spread']*data.iloc[-1,4]\
    +res2.params['int_risk']*data.iloc[-1,5]+res2.params['term_structure']*data.iloc[-1,6]+\
    res2.params['default_risk']*data.iloc[-1,7]+res2.params['house']*data.iloc[-1,8]+res2.params['Intercept']
    #print(Var_5)
    #第二次回归，因变量为金融system的收益率，也是为求出分位数回归的系数
    model_3=smf.quantreg('bank_sys_ret ~ retu + SH_retu + SH_vol + liquid_spread + int_risk + term_structure + default_risk + house',data)
    res3=model_3.fit(q=0.05)
    beta_sys_ret=res3.params['retu']
    #print(beta_sys_ret)
    delta_covar=beta_sys_ret*(Var_5-Var_50)
    return delta_covar
    
    

开始计算每个银行的$\Delta CoVaR$

In [281]:
bank_covar=[]

for k in range(bank_research_len):
    covar_record=[];date_record=[];
    bank_date_len=bank14_add_marco[k].shape[0] #记录每个银行日期的长度
    #整段跑时间太长了，要2个小时左右，可以调整下面的bank_date_len为18跑个小一点的子样本
    for i in range(15,bank_date_len):  #从15开始的原因是，分位数回归要估计8个系数，需要挺多样本的，避免出bug
        data=bank14_add_marco[k].iloc[:i,:]  
        #print(data_address(data))
        covar_value=covar(data)
        covar_record.append(covar_value)
        #print(data.iloc[-1,0])
        date_record.append(data.iloc[-1,0])
    #print(date_record)
    temp={'Date':date_record,'CoVaR':covar_record}
    temp_df=DataFrame(temp) #将字典类型的temp转换为dataframe类型
    bank_covar.append(temp_df)


导出$\Delta CoVaR$数据

In [282]:
#导出covar数据
#bank14_name

for i in range(bank_research_len):
    path_name='covar_value/'+bank14_name[i]+'.csv'
    bank_covar[i].to_csv(path_name)


In [283]:
print(bank14_add_marco[1]['bank_sys_ret'])

0     -0.018860
1      0.024397
2      0.084205
3     -0.039448
4     -0.018183
5     -0.048942
6      0.000572
7      0.038151
8     -0.052744
9      0.000177
10     0.020091
11    -0.007545
12     0.050057
13    -0.068355
14    -0.092858
15    -0.084461
16     0.056897
17    -0.040805
18    -0.054575
19     0.021247
20    -0.019941
21    -0.094561
22    -0.012915
23     0.018468
24     0.036108
25    -0.020512
26    -0.112075
27     0.135568
28     0.038535
29    -0.090577
         ...   
671    0.006644
672    0.043328
673   -0.010493
674    0.008077
675    0.047037
676    0.003431
677    0.005973
678   -0.015342
679    0.022197
680   -0.004886
681   -0.018610
682    0.000807
683    0.003870
684   -0.003707
685   -0.009288
686    0.004799
687   -0.023364
688    0.018199
689    0.011592
690   -0.017014
691    0.031302
692   -0.015933
693   -0.014728
694   -0.020574
695    0.015325
696   -0.019901
697   -0.036781
698   -0.014166
699   -0.017517
700   -0.048749
Name: bank_sys_ret, Leng